In [5]:
#Read commented Instructions 

#import required libraries for loading your dataset

import pandas as pd
import numpy as np
import json

In [2]:
with open("layer1.json") as datafile:
    data = json.load(datafile)
dataframe = pd.DataFrame(data)

In [3]:
import re 
A = []

for urls in dataframe['url']:
    domain = urls.split('/')[2]
    A.append(domain)

In [4]:
dataframe['Domain'] = A

In [6]:
df

,ingredients,url,partition,title,id,instructions,Domain
0,"[{'text': '6 ounces penne'}, {'text': '2 cups ...",http://www.epicurious.com/recipes/food/views/-...,train,Worlds Best Mac and Cheese,000018c8a5,[{'text': 'Preheat the oven to 350 F. Butter o...,www.epicurious.com
1,"[{'text': '1 c. elbow macaroni'}, {'text': '1 ...",http://cookeatshare.com/recipes/dilly-macaroni...,train,Dilly Macaroni Salad Recipe,000033e39b,[{'text': 'Cook macaroni according to package ...,cookeatshare.com
2,"[{'text': '8 tomatoes, quartered'}, {'text': '...",http://www.foodnetwork.com/recipes/gazpacho1.html,train,Gazpacho,000035f7ed,[{'text': 'Add the tomatoes to a food processo...,www.foodnetwork.com
3,"[{'text': '2 12 cups milk'}, {'text': '1 12 cu...",http://www.food.com/recipe/crunchy-onion-potat...,test,Crunchy Onion Potato Bake,00003a70b1,[{'text': 'Preheat oven to 350 degrees Fahrenh...,www.food.com
4,[{'text': '1 (3 ounce) package watermelon gela...,http://www.food.com/recipe/cool-n-easy-creamy-...,train,Cool 'n Easy Creamy Watermelon Pie,00004320bb,"[{'text': 'Dissolve Jello in boiling water.'},...",www.food.com
...,...,...,...,...,...,...,...
995,"[{'text': '4 Horse mackerel, large'}, {'text':...",https://cookpad.com/us/recipes/147822-oven-bak...,test,Oven-Baked Horse Mackerel,004060114d,[{'text': 'Cut into belly of the horse mackere...,cookpad.com
996,[{'text': '6 tablespoons Grade B maple syrup (...,http://www.epicurious.com/recipes/food/views/b...,train,Broiled Salmon with Spicy Maple Basting Sauce,0040747ee2,[{'text': 'In a small heavy saucepan combine m...,www.epicurious.com
997,"[{'text': '1 cup warm water'}, {'text': '12 te...",http://www.food.com/recipe/greek-cyprus-olive-...,val,Greek Cyprus Olive Garlic Bread With Lemon Zes...,0040770153,[{'text': 'Stir together the water and sugar i...,www.food.com
998,"[{'text': '1 tablespoon butter'}, {'text': '2 ...",http://www.food.com/recipe/snow-peas-with-fres...,train,Snow Peas With Fresh Mint,004080d77d,"[{'text': 'NOTE: for a more Asian flare, use a...",www.food.com


In [ ]:
#Scrapping code that you have to use, i used df['url'] change 'df' according to your requirement




from recipe_scrapers import scrape_me
import pandas as pd
import re
import requests
from bs4 import BeautifulSoup as bs
import numpy as np


Nutritions=[]





for urls in df['url']:
    
    
    
    if('cookeatshare.com' in urls) == True:
        
        try:
            
            header = {"User-Agent": "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/50.0.2661.75 Safari/537.36","X-Requested-With": "XMLHttpRequest"}
            r = requests.get(urls, headers=header)
            soup = bs(r.content,'html.parser')
            Nutrition_url = soup.find('div', {'id':'detailed_view'})
            for link in Nutrition_url.find_all('a', attrs={'href': re.compile("/")}):
                url = link.get('href')
    
            r = requests.get('https://cookeatshare.com'+url, headers=header)
            df_cook = pd.read_html(r.text)
            df_cook = df_cook[1]
            df_cook= df_cook[[df_cook.columns[0], 'Total for Recipe']] 
            nutrition = df_cook.set_index(df_cook.columns[0])['Total for Recipe'].to_dict()
            
            Nutritions.append(nutrition)
             

            
        except :
            
            Nutritions.append(np.nan)
            
            
            
    elif('allrecipes.com' in urls) == True:
        
        try:
            
            nutrition = []
            capitals = {}

            r = requests.get(urls)
            soup = bs(r.content,'html.parser')
            
            #cal value
            
            nutrition1 = soup.find('div',{'class':'partial recipe-nutrition-section'})
            nutrition_cal = nutrition1.find('div',{'class':'section-body'}).get_text()
            nutrition_cal= nutrition_cal.strip().split(';')

            pattern = '\d+\%|\d+\.\w+|\d+\w+'

            a =re.sub(pattern, ' ', nutrition_cal[0]).strip() 
            Result = re.search(pattern, nutrition_cal[0])
            b = Result.group()
            capitals[a] = b
            
            #other values
            
            
            nutriton = soup.find('div', {'class':'ng-dialog-content'})

            for values in nutriton.find_all('div',{'class':'nutrition-row'}):
                nutrition.append(values.find('span',{'class':'nutrient-name'}).get_text().strip())
                
            for elements in nutrition:
                a = elements.split(":")[0]
                b = elements.split(':')[1].strip()
                capitals[a] = b
                
            Nutritions.append(capitals)
            
            
        except:
            
            Nutritions.append(np.nan)
            
            
    elif ('www.myrecipes.com' in urls) == True:
        
        try:
            
            nutrition = []
            capitals = {}
            
            r = requests.get(urls)
            soup = bs(r.content,'html.parser')
            nutriton = soup.find('section', {'class':'nutrition-section container'})

            value = nutriton.find('div', {'class':'section-body'}).get_text()

            split = value.split(';')
            pattern = '\d+\%|\d+\.\w+|\d+\w+' 

            for elements in split:
                a =re.sub(pattern, ' ', elements).strip() 
                Result = re.search(pattern, elements)
                b = Result.group()
                capitals[a] = b
                
            Nutritions.append(capitals)
            
        except:
            
            Nutritions.append(np.nan)
            
    
    elif ('recipeland.com' in urls) == True:
        
        try:
            
            nutrition = []
            capitals = {}
            
            r = requests.get(urls)
            soup = bs(r.content,'html.parser')
            Nutrition = soup.find('div', {'class':'nfacts'}, id = 'nutrition')

            cal_value = Nutrition.find('div',{'class':'nut_tb'}, 'b').get_text()
            nutrition.append(cal_value)


            for i in range(0,5):
                values1 = Nutrition.find_all('div',{'class':'nut_top'})[i].get_text()
                nutrition.append(values1)

            for i in range(0,3):
                values2 = Nutrition.find_all('div',{'class':'indent_top'})[i].get_text()
                nutrition.append(values2)
    

            pattern2 = '\d+\%|\d+\.\w+|\d+\w+|(of calories from fat)' 
            pattern1 = '\d{1,2}\%' 
            ele = []

            for elements in nutrition:
                element = elements.strip()
                Result = re.sub(pattern1,' ' ,element).strip()
                ele.append(Result)


            for elements in ele:
                a =re.sub(pattern2, ' ', elements).strip() 
                Result = re.search(pattern2, elements)
                b = Result.group()
                capitals[a] = b
                
            Nutritions.append(capitals)
            
        except:
            
            Nutritions.append(np.nan)
            
            
    elif ('www.landolakes.com' in urls) == True:
        
        try:
            scraper = scrape_me(urls, wild_mode=True)
            nutrition = scraper.nutrients()
            Nutritions.append(nutrition)
            
        except:
            Nutritions.append(np.nan)
            
        
            
            
    else:
        
        try:
            scraper = scrape_me(urls)
            nutrition = scraper.nutrients()
            Nutritions.append(nutrition)
        
        except:
            
            Nutritions.append(np.nan)
        
        
        
            
                
            
            



In [ ]:
#Save the list as a dataframe


df_0_1000 = pd.DataFrame(Nutritions,columns=['Nutritions'])
df_0_1000.to_csv('Nutritions_0_1000')

In [10]:
len(Nutritions)

1000

In [16]:
#Check by doing this if length of your list is same as length of dataframe it will not throw any error

df['Nutritions'] = Nutritions

<ipython-input-16-3324af45dd83>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Nutritions'] = Nutritions


In [22]:
#Skip this

pd.set_option("display.max_rows", None, "display.max_columns", None)
df

,ingredients,url,partition,title,id,instructions,Domain,Nutritions
0,"[{'text': '6 ounces penne'}, {'text': '2 cups ...",http://www.epicurious.com/recipes/food/views/-...,train,Worlds Best Mac and Cheese,000018c8a5,[{'text': 'Preheat the oven to 350 F. Butter o...,www.epicurious.com,{}
1,"[{'text': '1 c. elbow macaroni'}, {'text': '1 ...",http://cookeatshare.com/recipes/dilly-macaroni...,train,Dilly Macaroni Salad Recipe,000033e39b,[{'text': 'Cook macaroni according to package ...,cookeatshare.com,"{'Protein (g)': 89.9, 'Total Fat (g)': 177.9, ..."
2,"[{'text': '8 tomatoes, quartered'}, {'text': '...",http://www.foodnetwork.com/recipes/gazpacho1.html,train,Gazpacho,000035f7ed,[{'text': 'Add the tomatoes to a food processo...,www.foodnetwork.com,{}
3,"[{'text': '2 12 cups milk'}, {'text': '1 12 cu...",http://www.food.com/recipe/crunchy-onion-potat...,test,Crunchy Onion Potato Bake,00003a70b1,[{'text': 'Preheat oven to 350 degrees Fahrenh...,www.food.com,"{'calories': '240.7', 'fatContent': '18', 'sat..."
4,[{'text': '1 (3 ounce) package watermelon gela...,http://www.food.com/recipe/cool-n-easy-creamy-...,train,Cool 'n Easy Creamy Watermelon Pie,00004320bb,"[{'text': 'Dissolve Jello in boiling water.'},...",www.food.com,"{'calories': '392.6', 'fatContent': '24.4', 's..."
5,"[{'text': '12 cup shredded coconut'}, {'text':...",http://www.food.com/recipe/easy-tropical-beef-...,train,Easy Tropical Beef Skillet,0000631d90,"[{'text': 'In a large skillet, toast the cocon...",www.food.com,"{'calories': '472.5', 'fatContent': '23.8', 's..."
6,"[{'text': '2 Chicken thighs'}, {'text': '2 tsp...",https://cookpad.com/us/recipes/150100-kombu-te...,train,Kombu Tea Grilled Chicken Thigh,000075604a,[{'text': 'Pierce the skin of the chicken with...,cookpad.com,{}
7,"[{'text': '6 -8 cups fresh rhubarb, or'}, {'te...",http://www.food.com/recipe/strawberry-rhubarb-...,train,Strawberry Rhubarb Dump Cake,00007bfd16,[{'text': 'Put ingredients in a buttered 9 x 1...,www.food.com,"{'calories': '431.8', 'fatContent': '12.6', 's..."
8,"[{'text': '8 ounces, weight Light Fat Free Van...",http://tastykitchen.com/recipes/breakfastbrunc...,train,Yogurt Parfaits,000095fc1d,[{'text': 'Layer all ingredients in a serving ...,tastykitchen.com,{}
9,"[{'text': '2 cups flour'}, {'text': '1 tablesp...",http://www.food.com/recipe/zucchini-nut-bread-...,train,Zucchini Nut Bread,0000973574,"[{'text': 'Sift dry ingredients.'}, {'text': '...",www.food.com,"{'calories': '683.3', 'fatContent': '39', 'sat..."


In [24]:
#again i tried same code on rows between 1000-2000

# keep this thing is mind- first i used my code on first 1000 and in second stage i used (1000-2000) as you know in slicing it does not consider your last index so keep this thing in mind

df1 = dataframe[1000:2000]
df1

,ingredients,url,partition,title,id,instructions,Domain
1000,[{'text': '3 (8 ounce) packages crescent rolls...,http://www.food.com/recipe/sausage-rolls-121361,train,Sausage Rolls,0040aa3b23,[{'text': 'Mix Fried sausage and rest of ingre...,www.food.com
1001,[{'text': '5 tablespoons extra virgin olive oi...,http://www.cookstr.com/recipes/bigoli-with-smo...,train,Bigoli with Smoked Sausage and Borlotti Beans,0040aeb722,[{'text': 'In a wide skillet large enough to a...,www.cookstr.com
1002,"[{'text': '20 cloves garlic, peeled'}, {'text'...",http://www.foodnetwork.com/recipes/michael-chi...,train,Prime Rib-Eye Steaks with Mustard Parmesan Crust,0040b5d4d2,[{'text': 'In a preheated saute pan over mediu...,www.foodnetwork.com
1003,"[{'text': '1 cup sliced fresh mushrooms'}, {'t...",http://www.kraftrecipes.com/recipes/pizza-pepp...,test,Pizza with Pepperoni and Veggies,0040c511dd,"[{'text': 'Heat oven to 450 degrees F.'}, {'te...",www.kraftrecipes.com
1004,"[{'text': '1/4 cup brown sugar'}, {'text': '1 ...",http://cooking.nytimes.com/recipes/12553,val,Stone-Fruit Chutney,0040d67b19,"[{'text': 'Combine fruit, brown sugar and wate...",cooking.nytimes.com
1005,"[{'text': '12 ounces pasta, i used rotelle'}, ...",http://www.food.com/recipe/pasta-with-creamy-r...,train,Pasta with Creamy Red Roasted Pepper Sauce,0040d6ad3c,[{'text': 'Cook pasta according to package dir...,www.food.com
1006,"[{'text': '2 cups flour'}, {'text': '2 teaspoo...",http://www.food.com/recipe/apple-banana-bread-...,train,Apple Banana Bread,0040e5a887,"[{'text': 'Preheat oven to 375 degrees.'}, {'t...",www.food.com
1007,[{'text': '3 cups coarsely shredded zucchini'}...,http://allrecipes.com/recipe/zucchini-cakes-3/,train,Zucchini Cakes,0040e8d42a,[{'text': 'Place grated zucchini in a colander...,allrecipes.com
1008,"[{'text': '6 Whole-wheat Pitas'}, {'text': '1/...",http://tastykitchen.com/recipes/appetizers-and...,train,Salted Whole Wheat Pita Chips,0040f98c36,[{'text': 'Preheat oven to 375 degrees F. Slic...,tastykitchen.com
1009,"[{'text': '1 sprig fresh lavender (garnish)'},...",http://www.food.com/recipe/citrusy-lavender-ci...,val,Citrusy Lavender Cider,0041022e38,"[{'text': 'Combine syrup, juice and alcohol in...",www.food.com


In [2]:
from recipe_scrapers import scrape_me
import pandas as pd
import re
import requests
from bs4 import BeautifulSoup as bs
import numpy as np


Nutritions=[]





for urls in df1['url']:
    
    
    
    if('cookeatshare.com' in urls) == True:
        
        try:
            
            header = {"User-Agent": "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/50.0.2661.75 Safari/537.36","X-Requested-With": "XMLHttpRequest"}
            r = requests.get(urls, headers=header)
            soup = bs(r.content,'html.parser')
            Nutrition_url = soup.find('div', {'id':'detailed_view'})
            for link in Nutrition_url.find_all('a', attrs={'href': re.compile("/")}):
                url = link.get('href')
    
            r = requests.get('https://cookeatshare.com'+url, headers=header)
            df_cook = pd.read_html(r.text)
            df_cook = df_cook[1]
            df_cook= df_cook[[df_cook.columns[0], 'Total for Recipe']] 
            nutrition = df_cook.set_index(df_cook.columns[0])['Total for Recipe'].to_dict()
            
            Nutritions.append(nutrition)
             

            
        except :
            
            Nutritions.append(np.nan)
            
            
            
    elif('allrecipes.com' in urls) == True:
        
        try:
            
            nutrition = []
            capitals = {}

            r = requests.get(urls)
            soup = bs(r.content,'html.parser')
            
            #cal value
            
            nutrition1 = soup.find('div',{'class':'partial recipe-nutrition-section'})
            nutrition_cal = nutrition1.find('div',{'class':'section-body'}).get_text()
            nutrition_cal= nutrition_cal.strip().split(';')

            pattern = '\d+\%|\d+\.\w+|\d+\w+'

            a =re.sub(pattern, ' ', nutrition_cal[0]).strip() 
            Result = re.search(pattern, nutrition_cal[0])
            b = Result.group()
            capitals[a] = b
            
            #other values
            
            
            nutriton = soup.find('div', {'class':'ng-dialog-content'})

            for values in nutriton.find_all('div',{'class':'nutrition-row'}):
                nutrition.append(values.find('span',{'class':'nutrient-name'}).get_text().strip())
                
            for elements in nutrition:
                a = elements.split(":")[0]
                b = elements.split(':')[1].strip()
                capitals[a] = b
                
            Nutritions.append(capitals)
            
            
        except:
            
            Nutritions.append(np.nan)
            
            
    elif ('www.myrecipes.com' in urls) == True:
        
        try:
            
            nutrition = []
            capitals = {}
            
            r = requests.get(urls)
            soup = bs(r.content,'html.parser')
            nutriton = soup.find('section', {'class':'nutrition-section container'})

            value = nutriton.find('div', {'class':'section-body'}).get_text()

            split = value.split(';')
            pattern = '\d+\%|\d+\.\w+|\d+\w+' 

            for elements in split:
                a =re.sub(pattern, ' ', elements).strip() 
                Result = re.search(pattern, elements)
                b = Result.group()
                capitals[a] = b
                
            Nutritions.append(capitals)
            
        except:
            
            Nutritions.append(np.nan)
            
    
    elif ('recipeland.com' in urls) == True:
        
        try:
            
            nutrition = []
            capitals = {}
            
            r = requests.get(urls)
            soup = bs(r.content,'html.parser')
            Nutrition = soup.find('div', {'class':'nfacts'}, id = 'nutrition')

            cal_value = Nutrition.find('div',{'class':'nut_tb'}, 'b').get_text()
            nutrition.append(cal_value)


            for i in range(0,5):
                values1 = Nutrition.find_all('div',{'class':'nut_top'})[i].get_text()
                nutrition.append(values1)

            for i in range(0,3):
                values2 = Nutrition.find_all('div',{'class':'indent_top'})[i].get_text()
                nutrition.append(values2)
    

            pattern2 = '\d+\%|\d+\.\w+|\d+\w+|(of calories from fat)' 
            pattern1 = '\d{1,2}\%' 
            ele = []

            for elements in nutrition:
                element = elements.strip()
                Result = re.sub(pattern1,' ' ,element).strip()
                ele.append(Result)


            for elements in ele:
                a =re.sub(pattern2, ' ', elements).strip() 
                Result = re.search(pattern2, elements)
                b = Result.group()
                capitals[a] = b
                
            Nutritions.append(capitals)
            
        except:
            
            Nutritions.append(np.nan)
            
            
    elif ('www.landolakes.com' in urls) == True:
        
        try:
            scraper = scrape_me(urls, wild_mode=True)
            nutrition = scraper.nutrients()
            Nutritions.append(nutrition)
            
        except:
            Nutritions.append(np.nan)
            
        
            
            
    else:
        
        try:
            scraper = scrape_me(urls)
            nutrition = scraper.nutrients()
            Nutritions.append(nutrition)
        
        except:
            
            Nutritions.append(np.nan)
        
        
        
            
                
            
            



In [26]:
Nutritions

[{'calories': '1018.4',
  'fatContent': '64.7',
  'saturatedFatContent': '24.7',
  'cholesterolContent': '287.4',
  'sodiumContent': '2184.2',
  'carbohydrateContent': '64.5',
  'fiberContent': '4.3',
  'sugarContent': '5.5',
  'proteinContent': '41.9'},
 {},
 {},
 nan,
 {},
 {'calories': '332.5',
  'fatContent': '1.6',
  'saturatedFatContent': '0.3',
  'cholesterolContent': '0',
  'sodiumContent': '1169.5',
  'carbohydrateContent': '67.2',
  'fiberContent': '3.8',
  'sugarContent': '2.3',
  'proteinContent': '11.8'},
 {'calories': '930.9',
  'fatContent': '47.4',
  'saturatedFatContent': '23.7',
  'cholesterolContent': '197.3',
  'sodiumContent': '754.9',
  'carbohydrateContent': '118.9',
  'fiberContent': '5.2',
  'sugarContent': '61.5',
  'proteinContent': '12.9'},
 nan,
 {},
 {'calories': '149.9',
  'fatContent': '0.1',
  'saturatedFatContent': '0',
  'cholesterolContent': '0',
  'sodiumContent': '17.2',
  'carbohydrateContent': '2.6',
  'fiberContent': '0.3',
  'sugarContent': '0.

In [ ]:
# just save this second list as  .csv

df_1000_2000 = pd.DataFrame(Nutritions,columns=['Nutritions'])
df_1000_2000.to_csv('Nutritions_1000_2000')

In [ ]:
#now Convert both of your.csv file in list

df_lis_1 = pd.read_csv('Nutritions_0_1000')
lis_0_1000 = df_lis_1['Nutritions'].tolist()

df_lis_2 = pd.read_csv('Nutritions_1000_2000')
lis_1000_2000 = df_lis_2['Nutritions'].tolist()





#after doing this you will get your lists now simply add them..



In [ ]:
#this will be your last list

Final_Nutritions = lis_0_1000 + lis_1000_2000 

In [ ]:
#Save the list as a dataframe


df_1000_2000 = pd.DataFrame(Final_Nutritions,columns=['Nutritions'])
df_1000_2000.to_csv('Nutritions_0_2000')

# You will have to send me this last file

In [ ]:
#if you need any help just contact me


In [2]:
#########

In [3]:
## food dataset 10 

In [4]:


import pandas as pd
import json
import numpy as np



In [5]:
#! pip install recipe_scrapers

In [6]:
from recipe_scrapers import scrape_me
import pandas as pd
import re
import requests
from bs4 import BeautifulSoup as bs
import numpy as np

In [7]:
# Importing data

layer1_food_dataset = pd.read_csv('layer1_food_dataset_10.csv')
layer1_food_dataset

,Unnamed: 0,ingredients,url,partition,title,id,instructions
0,101138,"[{'text': '1 cup unsalted butter (2 sticks)'},...",http://www.food.com/recipe/buttery-new-orleans...,test,Buttery New Orleans Shrimp,19112b9031,[{'text': 'Prepare all ingredients and have re...
1,101139,"[{'text': '6 tablespoons dark roast coffee, fi...",http://www.food.com/recipe/dark-roast-coffee-g...,train,Dark Roast Coffee Gelee,1911399616,[{'text': 'Brew ground coffee in filter style ...
2,101140,"[{'text': '34 cup white sugar'}, {'text': '1 (...",http://www.food.com/recipe/creamy-caramel-flan...,train,Creamy Caramel Flan,191145af7b,"[{'text': 'Preheat oven to 350 degrees F.'}, {..."
3,101141,[{'text': '1 (8 ounce) can frozen orange juice...,http://www.food.com/recipe/booboo-punch-spanis...,train,Booboo Punch/Spanish Punch,19114be902,[{'text': 'Mix 1/2 can frozen orange juice con...
4,101143,[{'text': '8 -10 medium dried guajillo chilies...,http://www.food.com/recipe/guajillo-spiked-por...,train,Guajillo-Spiked Pork-And-Potato Tacos,19117acda2,"[{'text': 'To make the chili sauce, in a dry h..."
...,...,...,...,...,...,...,...
4995,111436,[{'text': '1 12 cups chocolate wafer crumbs (a...,http://www.food.com/recipe/chocolate-peanut-bu...,train,Chocolate Peanut Butter Chess Pie,1b9a82fd50,"[{'text': 'Preheat oven to 325 degrees F.'}, {..."
4996,111437,"[{'text': '1 lb rigatoni pasta'}, {'text': '2 ...",http://www.food.com/recipe/pizza-dump-35090,train,Pizza Dump,1b9a8d0e4f,[{'text': 'Cook rigatoni as directed on box & ...
4997,111439,[{'text': '1 sheet pepperidge farm frozen puff...,http://www.food.com/recipe/chocolate-walnut-st...,train,Chocolate Walnut Strudel,1b9aa4571a,[{'text': 'Thaw sheet of puffed pastry for 30 ...
4998,111440,"[{'text': '2 12-3 12 cups flour'}, {'text': '2...",http://www.food.com/recipe/chewy-bread-rings-2...,test,Chewy Bread Rings,1b9aba8fb0,"[{'text': 'Grease 2 cookie sheets.'}, {'text':..."


In [ ]:
%%time
Nutritions=[]

for url,title in layer1_food_dataset[['url','title']][0:2000].itertuples(index=False):
  try:
      scraper = scrape_me(url)
      nutrition = scraper.nutrients()
      nutrition['title'] = title
      Nutritions.append(nutrition)

  except:
      
      Nutritions.append(np.nan)



In [ ]:
Nutritions

In [ ]:
layer1_food_dataset[['url','title']][0:2000]

In [ ]:
# just save this second list as  .csv

df_0_2000 = pd.DataFrame(Nutritions,columns=['Nutritions'])
df_0_2000.to_csv('Nutritions_0_2000')

In [ ]:
%%time
Nutritions=[]

for url,title in layer1_food_dataset[['url','title']][2000:4000].itertuples(index=False):
  try:
      scraper = scrape_me(url)
      nutrition = scraper.nutrients()
      nutrition['title'] = title
      Nutritions.append(nutrition)

  except:
      
      Nutritions.append(np.nan)



In [ ]:
layer1_food_dataset[['url','title']][3000:5000]

In [ ]:
# just save this second list as  .csv

df_3000_5000 = pd.DataFrame(Nutritions,columns=['Nutritions'])
df_3000_5000.to_csv('Nutritions_3000_5000')